# Projeto: Deploy em Produção de Projeto de Machine Learning

**Degree**: Data Science | **Turma**: 815 | **Instituição**: Let's Code by Ada | **Módulo**: XVI - Modelos Produtivos I

Professor: [Davi Nascimento](https://github.com/davicn)

### Grupo

**Colegas participantes do grupo para o projeto**:


* [Ana Gabriela de Castro Almeida](https://github.com/almeidacastrogabriela)

* [Douglas Alves](https://github.com/dougalve)

* [Jeremias Diefenthaler](https://github.com/JeremiasDief)

------------------------
--------------------

### Contexto

O dataset original possui 1.000 entradas com 20 atributos categóricos e/ou símbolos preparados pelo Professor Hofmann da UCI.
Neste dataset, cada entrada representada uma pessoa que solicita crédito em um banco. Cada pessoa é classificada com risco de crédito bom ou ruim de acordo com o conjunto de atributos.

### Conteúdo

O dataset foi adquirido no [Kaggle](https://www.kaggle.com/datasets/uciml/german-credit).
E os atributos selecionados são:

| Atributo em Português | Nome Original Atributo | Tipo |
|---------------------- | -----------------------|------|
| Idade                 | Age                    | Numérica|
| Gênero | Sex | Texto: male (homem), female (mulher)|
| Empregabilidade| Job | Numérica: 0 - unskilled and non-resident (sem qualificação e não residente) 1 - unskilled and resident (sem qualificação e residente) 2 - skilled (qualificado) e 3 - highly skilled (altamente qualificado)|
| Habitação | Housing | Texto: own (própria), rent (alugada), free (gratuita) |
| Conta Poupança | Saving accounts| Text: little (pouco), moderate (moderado), quite rich (bem rico) e rich (rico)|
| Conta Corrente | Checking account | Numérica (em Marcação Alemã) |
| Quantia de Crédito | Credit amount | Numérica (em Marcação Alemã) |
| Duração | Duration | Numérica (em Meses) |
| Motivo  |Purpose |Text: car (carro), furniture/equipment (móveis/equipamentos), radio/, domestic appliances (eletrodomésticos), repairs (reparos), education (educação), business (negócios), vacation/others (viagens/outros) |

### Objetivo do Projeto


Ao longo do curso de Data Science, aprendemos diversas técnicas de Aprendizado de Máquina e seguindo as regras de Interpretabilidade de Modelos, definimos qual o melhor modelo de Machine Learning para subir em produção.

Com base nisso, implementamos as técnicas vistas no módulo final do curso, **Modelos Produtivos I**, como o MLFlow e por fim, fizemos o deploy em produção para consulta do modelo utilizando nossa API em FastAPI, seguindo a disponibilização do nosso localhost pelo ***ngrok***.

In [3]:
# Importando as bibliotecas necessárias

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from ml_utils import *
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix)
import mlflow

In [4]:
# Lendo o dataset

df = pd.read_csv("german_credit_data.csv", index_col=0)
df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [5]:
# Verificando o tipo de cada coluna do dataset

df.dtypes

Age                  int64
Sex                 object
Job                  int64
Housing             object
Saving accounts     object
Checking account    object
Credit amount        int64
Duration             int64
Purpose             object
Risk                object
dtype: object

In [6]:
# Verificando quais colunas possuem dados nulos

df.isnull().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [7]:
# Preenchendo os dados faltantes nas colunas "Saving accounts" e "Checking account" com o valor "empty" (vazio)

df[["Saving accounts", "Checking account"]] = df[["Saving accounts", "Checking account"]].fillna("empty")

In [8]:
# Verificando o dataset após o preenchimento dos valores nulos

df

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,empty,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,empty,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,empty,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,empty,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


In [7]:
# Conferindo que não existe mais nenhuma coluna com algum dado faltante

df.isnull().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In [8]:
# Separando a coluna target do restante do dataset, atribuindo as variáveis X e y
X = df.drop(columns="Risk")
y = df["Risk"]

# Transformando a saída da coluna target ("Risk") para números: "bad"=0 e "good"=1
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

# Transformando as colunas categóricas para colunas numéricas, atribuindo um número para cada categoria
features_cat = X.select_dtypes(exclude=np.number).columns.tolist()
ordenc = OrdinalEncoder()
ordenc.fit(X[features_cat])
X[features_cat] = ordenc.transform(X[features_cat])

# Separando o dataset em treino e teste, com test_size de 20% e estratificando pela coluna target (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [25]:
# Verificando as categorias de cada coluna categórica (após a transformação, cada categoria recebeu um valor de 0 a n-1)

ordenc.categories_

[array(['female', 'male'], dtype=object),
 array(['free', 'own', 'rent'], dtype=object),
 array(['empty', 'little', 'moderate', 'quite rich', 'rich'], dtype=object),
 array(['empty', 'little', 'moderate', 'rich'], dtype=object),
 array(['business', 'car', 'domestic appliances', 'education',
        'furniture/equipment', 'radio/TV', 'repairs', 'vacation/others'],
       dtype=object)]

In [9]:
# Verificando os dados de X_train após todas as transformações

X_train

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
675,26,0.0,3,2.0,1.0,0.0,4530,30,5.0
703,41,1.0,2,1.0,2.0,2.0,2503,30,0.0
12,22,0.0,2,1.0,1.0,2.0,1567,12,5.0
845,35,1.0,2,1.0,0.0,2.0,3976,21,4.0
795,22,0.0,2,2.0,2.0,0.0,2301,9,4.0
...,...,...,...,...,...,...,...,...,...
284,37,1.0,2,1.0,2.0,2.0,3878,24,1.0
169,31,1.0,2,1.0,1.0,2.0,1935,24,0.0
856,40,0.0,2,1.0,0.0,0.0,894,10,3.0
655,22,1.0,2,0.0,1.0,1.0,3973,14,1.0


In [10]:
# Verificando quais colunas são numéricas (após a transformação, todas viraram numéricas)

X_train.select_dtypes(include=np.number).columns.tolist()

['Age',
 'Sex',
 'Job',
 'Housing',
 'Saving accounts',
 'Checking account',
 'Credit amount',
 'Duration',
 'Purpose']

In [11]:
# Verificando quais colunas são categóricas (como previsto, nenhuma coluna continuou categórica)

X_train.select_dtypes(exclude=np.number).columns.tolist()

[]

In [15]:
# Construindo um Pipeline para testar um modelo com os nossos dados transformados

pipe_svc = Pipeline([("scaler", StandardScaler()),
                     ("svc", SVC(probability=True, random_state=42))])

pipe_svc.fit(X_train, y_train)

y_pred = pipe_svc.predict(X_test)

In [16]:
# Função que retornará as métricas: acurácia, precision, recall e f1_score da nossa predição

def get_metrics(y_test:list, y_pred:list) -> list:
    ac = accuracy_score(y_test, y_pred)
    pr = precision_score(y_test, y_pred, average='weighted')
    rc = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return [ac, pr, rc, f1]

In [17]:
# Testando a função com o modelo de teste

get_metrics(y_test, y_pred)

[0.715, 0.6836895111766151, 0.715, 0.6681083957280225]

------------------------------------------------

### Integração com o MLflow

Abaixo, o código utilizando em um terminal Linux para fazer a conexão com o mlflow:

```mlflow server --backend-store-uri sqlite:///mlrunsdb.db \
              --default-artifact-root ./mlflowruns \
              --host 127.0.0.1 \
              --port 8000```

In [19]:
# Conectando um banco de dados sqlite ao mlflow, para podermos rodar nossos experimentos

DB_URI = 'sqlite:///mlrunsdb.db'
mlflow.set_tracking_uri(DB_URI)

tags = {
    "Módulo":"Modelos Produtivos 1",
    "Turma":815,
    "objeto":'ProjetoFinal'
}
mlflow.set_experiment(experiment_name='Classificação German Credit - Projeto Final')
mlflow.set_experiment_tags(tags=tags)

In [20]:
### PRIMEIRO EXPERIMENTO:
## Modelo: SVC
## Parâmetro base: kernel="rbf"

# Abaixo os passos que o mlflow irá fazer para calcular as métricas do modelo
# Cada experimento fica salvo dentro da conexão com o MLflow, na aba "Experiments"

with mlflow.start_run(
    run_name="API_Projeto",
    description="Classificação German Credit",
    tags={"version": "v1", "env": "dev"}
) as model_run:
        
    kernel = "rbf"

    pipe_svc = Pipeline([("scaler", StandardScaler()),
                         ("svc", SVC(probability=True, random_state=42))])
    
    pipe_svc.fit(X_train, y_train)

    y_pred = pipe_svc.predict(X_test)
    
    acuracia, precision, recall, f1 = get_metrics(y_test, y_pred)
       
    params = {"kernel": kernel,
              "size_train_dataset": len(X_train), 
              "size_test_dataset": len(X_test)}
    
    mlflow.log_params(params=params)
    
    metrics = {"acuracia": acuracia,
               "precision": precision,
               "recall": recall,
               "f1_score": f1}
    
    mlflow.log_metrics(metrics=metrics)
    
    mlflow.sklearn.log_model(pipe_svc, "model")

In [23]:
### SEGUNDO EXPERIMENTO:
## Modelo: KNN
## Parâmetro base: k=7

# Abaixo os passos que o mlflow irá fazer para calcular as métricas do modelo
# Cada experimento fica salvo dentro da conexão com o MLflow, na aba "Experiments"

with mlflow.start_run(
    run_name="API_Projeto",
    description="Classificação German Credit",
    tags={"version": "v2", "env": "dev"}
) as model_run:
        
    k = 7

    pipe_knn = Pipeline([("scaler", StandardScaler()),
                         ("knn", KNeighborsClassifier(n_neighbors=k))])
    
    pipe_knn.fit(X_train, y_train)

    y_pred = pipe_knn.predict(X_test)
    
    acuracia, precision, recall, f1 = get_metrics(y_test, y_pred)
       
    params = {"k": k,
              "size_train_dataset": len(X_train), 
              "size_test_dataset": len(X_test)}
    
    mlflow.log_params(params=params)
    
    metrics = {"acuracia": acuracia,
               "precision": precision,
               "recall": recall,
               "f1_score": f1}
    
    mlflow.log_metrics(metrics=metrics)
    
    mlflow.sklearn.log_model(pipe_knn, "model")

In [24]:
### TERCEIRO EXPERIMENTO:
## Modelo: RandomForest
## Parâmetro base: max_depth=8

with mlflow.start_run(
    run_name="API_Projeto",
    description="Classificação German Credit",
    tags={"version": "v3", "env": "dev"}
) as model_run:
        
    max_depth = 8

    pipe_rf = Pipeline([("scaler", StandardScaler()),
                         ("rf", RandomForestClassifier(max_depth=8, random_state=42))])
    
    pipe_rf.fit(X_train, y_train)

    y_pred = pipe_rf.predict(X_test)
    
    acuracia, precision, recall, f1 = get_metrics(y_test, y_pred)
       
    params = {"max_depth": max_depth,
              "size_train_dataset": len(X_train), 
              "size_test_dataset": len(X_test)}
    
    mlflow.log_params(params=params)
    
    metrics = {"acuracia": acuracia,
               "precision": precision,
               "recall": recall,
               "f1_score": f1}
    
    mlflow.log_metrics(metrics=metrics)
    
    mlflow.sklearn.log_model(pipe_rf, "model")

### Integração com o FastAPI

Abaixo, o código utilizado em um terminal Linux para fazer a integração com o FastAPI:

```uvicorn api-projeto:app --reload --port 8001```

 - Obs.: os próximos passos foram testando a api-projeto dentro do link gerado pelo FastAPI

### Gerando um link para acesso externo com o Ngrok

Como todos os comandos e testes foram feitos em um localhost, foi preciso gerar um link externo para acesso de terceiros utilizando o ngrok. Abaixo, o código utilizando em um terminal Linux para geração de um link com ngrok:

```ngrok http 8001```